In [0]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint

In [83]:
data2=pd.read_csv('winequality-red.csv', sep=';') #importing the data
n=len(data2.axes[0]) #to find number of rows in each attribute
print('Number of rows are: ', n)
for i in range(n):
  x=data2.iloc[i,-1] #for 'quality' attribute
  
  if(x<5):
    x=0
  elif(x==6 or x==5):
    x=1
  else:
    x=2
  data2.iloc[i,-1]=x

# for making four BINs
for i in data2:
  if (i=='quality'):
    break
  assign=pd.cut(data2[i],4,labels=['0','1','2','3'],precision=100)
  data2.drop( [i],inplace=True, axis=1 )
  data2.insert(0,i,assign)

print(data2.iloc[0:20,:])

Number of rows are:  1599
   alcohol sulphates pH  ... volatile acidity fixed acidity quality
0        0         0  2  ...                1             0       1
1        0         0  1  ...                2             1       1
2        0         0  1  ...                1             1       1
3        0         0  1  ...                0             2       1
4        0         0  2  ...                1             0       1
5        0         0  2  ...                1             0       1
6        0         0  1  ...                1             1       1
7        0         0  2  ...                1             0       2
8        0         0  1  ...                1             1       2
9        1         1  1  ...                1             1       1
10       0         0  1  ...                1             0       1
11       1         1  1  ...                1             1       1
12       0         0  2  ...                1             0       1
13       0         2  

In [84]:
dataset =data2 


def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

def InfoGain(data,split_attribute_name,target_name="quality"): 
    elements,counts = np.unique(data[target_name],return_counts = True)
    total_entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))]) #Calculate the entropy of the total dataset
    #total_entropy = entropy(data[target_name]) 
    vals,counts= np.unique(data[split_attribute_name],return_counts=True)
    Weighted_Entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])

    Information_Gain = total_entropy - Weighted_Entropy    # the information gain value
    return Information_Gain


def ID3(data,originaldata,features,target_attribute_name="quality",parent_node_class = None):
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    elif len(data)<10:
        return np.unique(originaldata[target_attribute_name])[np.argmax(np.unique(originaldata[target_attribute_name],return_counts=True)[1])]
    
    elif len(features)==0: # stop splitting if it has value less than 10
        return parent_node_class
    
    #If none of the above holds true, grow the tree!
    
    else:
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name],return_counts=True)[1])]        #Set the default value for this node --> The mode target feature value of the current node
        
        #Select the feature which best splits the dataset
        item_values = [InfoGain(data,feature,target_attribute_name) for feature in features] #Return the information gain values for the features in the dataset
        best_feature_index = np.argmax(item_values)
        best_feature = features[best_feature_index]
        tree = {best_feature:{}}
        features = [i for i in features if i != best_feature]        #Remove the feature with the best inforamtion gain from the feature space
        
        for value in np.unique(data[best_feature]):   # a branch under the root node for each possible value of the root node feature
            value = value
            sub_data = data.where(data[best_feature] == value).dropna()
            subtree = ID3(sub_data,dataset,features,target_attribute_name,parent_node_class)
            tree[best_feature][value] = subtree
            
        return(tree)    


def predict(query,tree,default = 1):    # to find predicted value
    for key in list(query.keys()):
        if key in list(tree.keys()):
            try:
                result = tree[key][query[key]] 
            except:
                return default
            result = tree[key][query[key]]  
            if isinstance(result,dict):
                return predict(query,result)
            else:
                return result


training_data = dataset.iloc[:1200].reset_index(drop=True)  #We drop the index respectively relabel the index
testing_data = dataset.iloc[1200:].reset_index(drop=True)    #starting form 0, because we do not want to run into errors regarding the row labels / indexes


def test(data,tree):
    queries = data.iloc[:,:-1].to_dict(orient = "records")      # Create new query instances by simply removing the target feature column from the original dataset and convert it to a dictionary
    predicted = pd.DataFrame(columns=["predicted"])    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    
    #Calculate the prediction accuracy
    for i in range(len(data)):
        predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
    print('The accuracy of Decision tree model is: ',(np.sum(predicted["predicted"] == data["quality"])/len(data))*100,'%')

tree = ID3(training_data,training_data,training_data.columns[:-1])
pprint(tree)

test(testing_data,tree)
'''queries = testing_data.iloc[:,:-1].to_dict(orient = "records")      # Create new query instances by simply removing the target feature column from the original dataset and convert it to a dictionary
predicted = pd.DataFrame(columns=["predicted"])    #Create a empty DataFrame in whose columns the prediction of the tree are stored
    
#Calculate the prediction accuracy
for i in range(len(testing_data)):
  predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) 
  print('The accuracy of Decision tree model is: ',(np.sum(predicted["predicted"] == testing_data["quality"])/len(testing_data))*100,'%')'''

{'alcohol': {'0': {'fixed acidity': {'0': {'volatile acidity': {'0': {'free sulfur dioxide': {'0': 1.0,
                                                                                              '1': {'citric acid': {'0': 1,
                                                                                                                    '1': 1.0}},
                                                                                              '2': 1.0,
                                                                                              '3': 1.0}},
                                                                '1': {'density': {'1': {'pH': {'0': 1.0,
                                                                                               '1': {'total sulfur dioxide': {'0': {'sulphates': {'0': {'free sulfur dioxide': {'0': {'chlorides': {'0': {'citric acid': {'0': {'residual sugar': {'0': 1.0}},
                                                                          

'queries = testing_data.iloc[:,:-1].to_dict(orient = "records")      # Create new query instances by simply removing the target feature column from the original dataset and convert it to a dictionary\npredicted = pd.DataFrame(columns=["predicted"])    #Create a empty DataFrame in whose columns the prediction of the tree are stored\n    \n#Calculate the prediction accuracy\nfor i in range(len(testing_data)):\n  predicted.loc[i,"predicted"] = predict(queries[i],tree,1.0) \n  print(\'The accuracy of Decision tree model is: \',(np.sum(predicted["predicted"] == testing_data["quality"])/len(testing_data))*100,\'%\')'

# Part 2: using scikit learn classifier

In [85]:

from sklearn.tree import DecisionTreeClassifier
 
#initialization 
dataset = data2
train_features = dataset.iloc[:1200,:-1]
test_features = dataset.iloc[1200:,:-1]
train_targets = dataset.iloc[:1200,-1]
test_targets = dataset.iloc[1200:,-1]

tree = DecisionTreeClassifier(criterion = 'entropy').fit(train_features,train_targets)

y_pred_sklearn = tree.predict(test_features)

print("The prediction  using Scikit learn model is: ",tree.score(test_features,test_targets)*100,"%")

The prediction  using Scikit learn model is:  81.70426065162907 %


# Part 3: Cross validation


In [86]:
from sklearn import tree
from sklearn.model_selection import cross_val_score
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,accuracy_score , precision_score , recall_score
model=DecisionTreeClassifier(criterion='entropy', max_depth=4,min_samples_split=10)
df=data2
x1 = df.iloc[:,0:12]
y1 = df.iloc[:,-1]
model.fit(x1,y1)
y_predict=model.predict(x1)
print('accuracy score=', accuracy_score(y1,y_predict)*100)
print('precision score=', precision_score(y1,y_predict, average='macro'))
print('recall score=', recall_score(y1,y_predict, average='macro'))

print('crossvalidation score=', cross_val_score(model, x1,y1,cv=3).mean())

'''df = data2

x = df.iloc[:,0:12]
y = df.iloc[:,-1]

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=0)
depth = []
for i in range(len(y)):
    clf = tree.DecisionTreeClassifier(max_depth=i)
    # Perform 7-fold cross validation 
    scores = cross_val_score(estimator=clf, X=x, y=y, cv=3, n_jobs=4)
    depth.append((i,scores.mean()))
print(depth)'''

accuracy score= 100.0
precision score= 1.0
recall score= 1.0
crossvalidation score= 1.0


'df = data2\n\nx = df.iloc[:,0:12]\ny = df.iloc[:,-1]\n\nx_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.4,random_state=0)\ndepth = []\nfor i in range(len(y)):\n    clf = tree.DecisionTreeClassifier(max_depth=i)\n    # Perform 7-fold cross validation \n    scores = cross_val_score(estimator=clf, X=x, y=y, cv=3, n_jobs=4)\n    depth.append((i,scores.mean()))\nprint(depth)'

In [87]:
#precision recall for Decision tree with scikit learn

from sklearn.metrics import precision_recall_fscore_support as score


precision, recall, fscore, support = score(test_targets, y_pred_sklearn) # y_pred is value from logistic regression classifier

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))

precision: [0.36363636 0.87327824 0.2       ]
recall: [0.17391304 0.92690058 0.14705882]
fscore: [0.23529412 0.89929078 0.16949153]
